<a href="https://colab.research.google.com/github/ANKUSHSINGH-PAT/-Hostel-Management-System-/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing required library**




In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

**Importing file from drive**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_fake='/content/drive/MyDrive/Colab Notebooks/Fake.csv'          # Inserting fake and real dataset
path_true='/content/drive/MyDrive/Colab Notebooks/True.csv'
df_fake=pd.read_csv(path_fake)
df_true=pd.read_csv(path_true)

In [58]:
df_true.tail(5)

,title,text,subject,date,class
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1


In [59]:
df_fake.tail(5)

,title,text,subject,date,class
23466,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,Middle-east,"January 22, 2016",0
23467,Oregon Governor Says Feds ‘Must Act’ Against P...,"21st Century Wire says So far, after nearly 20...",Middle-east,"January 21, 2016",0
23468,Ron Paul on Burns Oregon Standoff and Jury Nul...,21st Century Wire says If you ve been followin...,Middle-east,"January 21, 2016",0
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,Tune in to the Alternate Current Radio Network...,Middle-east,"January 20, 2016",0
23470,Arizona Rancher Protesting in Oregon is Target...,RTOne of the most visible members of the armed...,Middle-east,"January 20, 2016",0


**Inserting a column called "class" for fake and real news dataset to categories fake and true news.**

In [6]:
df_fake["class"] = 0
df_true["class"] = 1

In [7]:
df_fake.shape,df_true.shape

((23481, 5), (21417, 5))

**Removing last 10 rows from both the dataset, for manual testing**

In [8]:
df_fake_manual_testing=df_fake.tail(10)
for i in range(23480,23470,-1):
  df_fake.drop([i], axis=0, inplace=True)  #delete the records row_wise
df_true_manual_testing=df_true.tail(10)
for i in range(21416,21406,-1):
  df_true.drop([i], axis=0, inplace=True)  #delete the records row_wise



**Merging the manual testing dataframe in single dataset and save it in a csv file**

In [9]:
df_manual_testing=pd.concat([df_fake_manual_testing,df_true_manual_testing],axis=0)
df_manual_testing.to_csv("manual_testing")

**Merging the main fake and true dataframe and "title", "subject" and "date" columns is not required for detecting the fake news, so I am going to drop the columns.**

In [10]:
df_merge=pd.concat([df_fake,df_true],axis=0)
df=df_merge.drop(["title","subject","date"],axis=1)
df.head(10)

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


**Randomly shuffling the dataframe**

In [23]:
df = df.sample(frac = 1)
df.tail(10)

,text,class
6081,ken ham is a right wing young earth creationi...,0
975,washington reuters the investigation into ...,1
15709,beirut reuters lebanese president michel a...,1
20992,only months ago obama s attorney general lor...,0
2439,reuters transcanada corp canada s no pip...,1
16086,deep state is trying to take down a president ...,0
9882,washington reuters the senate on tuesday o...,1
16798,tokyo reuters a typhoon roared towards jap...,1
17804,a spokesperson for the clinton foundation clai...,0
21456,andrew breitbart got under the skin of the lef...,0


In [22]:
df.isnull().sum()

text     0
class    0
dtype: int64

**Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.**

In [21]:
def word_drop(text):                                   
  text = text.lower()
  text = re.sub('\\[.*?\\]', '', text)
  text = re.sub("\\W"," ", text)
  text = re.sub('https?://\\S+|www\\.\\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\\n', '', text)
  text = re.sub('\\w*\\d\\w*', '', text)
  return text

In [12]:
df["text"]=df["text"].apply(word_drop)
df.head(10)

,text,class
0,donald trump just couldn t wish all americans ...,0
1,house intelligence committee chairman devin nu...,0
2,on friday it was revealed that former milwauk...,0
3,on christmas day donald trump announced that ...,0
4,pope francis used his annual christmas day mes...,0
5,the number of cases of cops brutalizing and ki...,0
6,donald trump spent a good portion of his day a...,0
7,in the wake of yet another court decision that...,0
8,many people have raised the alarm regarding th...,0
9,just when you might have thought we d get a br...,0


**Defining dependent and independent variable as x and y**

In [13]:
x=df["text"]
y=df["class"]

**Splitting the dataset into training set and testing set.**

In [18]:
x_test,x_train,y_test,y_train=train_test_split(x,y,test_size=.25)   

**Convert text to vectors**

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer         


In [20]:
vectorization= TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

**Implementation Of Logistic Regression**

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
LR=LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [26]:
LR.score(xv_test,y_test)

0.978846039574544

In [27]:
predict_LR=LR.predict(xv_test)

In [28]:
print(classification_report(y_test,predict_LR))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     17625
           1       0.98      0.98      0.98     16033

    accuracy                           0.98     33658
   macro avg       0.98      0.98      0.98     33658
weighted avg       0.98      0.98      0.98     33658



**Implementation Of Decision Tree Classifier**

In [31]:
from sklearn.tree import DecisionTreeClassifier

In [32]:
DT=DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [33]:
DT.score(xv_test,y_test)

0.9925723453562303

In [36]:
predict_DT=DT.predict(xv_test)

In [35]:
print(classification_report(y_test,predict_DT))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     17625
           1       0.99      0.99      0.99     16033

    accuracy                           0.99     33658
   macro avg       0.99      0.99      0.99     33658
weighted avg       0.99      0.99      0.99     33658



**Implementation Of Random Forest Classifier**

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
RFC=RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [39]:
RFC.score(xv_test,y_test)

0.9845801889595341

In [40]:
predict_RFC=RFC.predict(xv_test)

In [41]:
print(classification_report(y_test,predict_RFC))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     17625
           1       0.98      0.98      0.98     16033

    accuracy                           0.98     33658
   macro avg       0.98      0.98      0.98     33658
weighted avg       0.98      0.98      0.98     33658



**Manual Testing**

In [55]:
def output_lable(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(word_drop) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]),  
                                                                                                              output_lable(pred_RFC[0])))

In [57]:
news = str(input())
manual_testing(news)

21413,LexisNexis withdrew two products from Chinese market,"LONDON (Reuters) - LexisNexis, a provider of legal, regulatory and business information, said on Tuesday it had withdrawn two products from the Chinese market in March this year after it was asked to remove some content.  The issue of academic freedom in China hit the headlines this week after the leading British academic publisher,  Cambridge University Press, said it had complied with a request to block online access to some scholarly articles in China.  It later reversed its position.   Earlier this year LexisNexis Business Insight Solutions in China was asked to remove some content from its database,  LexisNexis said in a statement.  In March 2017, the company withdrew two products (Nexis and LexisNexis Academic) from the Chinese market.  LexisNexis is owned by information group Relx. ",worldnews,"August 22, 2017 "


LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
RFC Prediction: Not A Fake News
